In [1]:
import matplotlib
matplotlib.use('nbagg')

import fiona
import numpy as np
from pathlib import Path
import os

import rasterio as rio
from rasterio import windows
from rasterio import features
import rasterio.plot as rioplot
from rasterio.warp import transform_bounds

from itertools import product
from rtree import index

from shapely.geometry import mapping, LineString, shape, Polygon, box
from shapely import wkt
import json

import matplotlib.pyplot as plt
#%matplotlib inline 


import geopandas as gp
import pandas as pd
import osmnx as ox

from functools import partial
import pyproj 
from shapely.ops import transform

import matplotlib.pyplot as plt
import matplotlib.colors as colors

#os_unpaved = ['service', 'residential', 'track', 'unclassified', 'living_street']
os_paved = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'primary_link', 'secondary_link', 'tertiary_link']


/home/lisa/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
DST_CRS    = 'EPSG:4326'
BORNEO_CRS = 'EPSG:32650'

DATADIR = "/home/lisa/Schreibtisch/Projekt/roadDetection/data/"

BOXDIR = DATADIR + "bounding_boxes/"
LABDIR = DATADIR + "OSM_extract/"

In [3]:
def clean_streets(allroads):
    if 'tag_0' not in allroads:
        tags = allroads['highway'].apply(pd.Series)
        tags = tags.rename(columns = lambda x : 'tag_' + str(x))
        newlines = pd.concat([allroads[:], tags[:]], axis=1)
    else:
        newlines = allroads
    newlines = newlines
    newlines.crs = {'init' : BORNEO_CRS}
    return newlines.to_crs({'init': DST_CRS})

In [4]:
def label_roads(allroads):
    allroads.name                                            = 'transformed_label_2'
    allroads.loc[allroads['tag_0'].isin(os_paved), 'name']   = 'transformed_label_1'
    allroads['label']                                        = 2
    allroads.loc[allroads['tag_0'].isin(os_paved), 'label']  = 1
    return allroads

In [5]:
def extract_streets(gdf):
    G = ox.graph_from_polygon(Polygon(gdf.boundary[0]), network_type='all')
    G_projected  = ox.project_graph(G)
    nodes, lines = ox.graph_to_gdfs(G_projected)
    clean_lines  = clean_streets(lines)
    return label_roads(clean_lines)

In [6]:
def save_roads(dst_dir, dst_name, roads):
    roads.to_file(dst_dir + dst_name + '.shp', driver='ESRI Shapefile')
    with open(dst_dir + dst_name + '.geojson', 'w') as f:
        f.write(roads.to_json())

In [8]:

for file in Path(BOXDIR).iterdir():
    if file.name.endswith(('.geojson')):
        regId = (file.name.rsplit('_')[1]).split('.')[0]
        outputName = 'RoadLabels_RDT_' + regId + '_OSM'

        gdf = gp.read_file(BOXDIR + file.name)
        lines = extract_streets(gdf)
        lines = lines.loc[:, ['geometry', 'name', 'label', 'tag_0']]
        save_roads(LABDIR, outputName, lines)
